# Análise de Sentimento - Reviews Mercado Livre (Dataset 1)

## Objetivo
Este notebook tem como objetivo realizar uma análise exploratória e de sentimento sobre o dataset `reviews_mercadolivre_com_br_1.json`.
Vamos investigar a distribuição das notas, padrões textuais e extrair insights sobre a satisfação dos clientes.

## Dataset
O dataset contém reviews de produtos extraídos do site Mercado Livre, com informações de data, nota (rating), texto do comentário e URL do produto.
O arquivo analisado é o `reviews_mercadolivre_com_br_1.json`.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_json('../datasets/reviews_mercadolivre_com_br_1.json')
print('Dataset foi inicializado!')
display(df.head())
# print(df.head())

Dataset foi inicializado!


,date,rating,content,product_url
0,09 set. 2023,5,Top.,https://produto.mercadolivre.com.br/MLB-314957...
1,19 ago. 2024,5,"Produto bom, cumpre o que promete.",https://produto.mercadolivre.com.br/MLB-314957...
2,15 fev. 2025,5,Ótima qualidade.,https://produto.mercadolivre.com.br/MLB-314957...
3,11 fev. 2025,4,Bom.,https://produto.mercadolivre.com.br/MLB-314957...
4,10 jan. 2025,5,Atendeu minhas expectativas.,https://produto.mercadolivre.com.br/MLB-314957...


In [12]:
print(f'Dimensçoes {df.shape}\n')
print(df.info())

Dimensçoes (103476, 4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103476 entries, 0 to 103475
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   date         103476 non-null  object
 1   rating       103476 non-null  int64 
 2   content      103476 non-null  object
 3   product_url  103476 non-null  object
dtypes: int64(1), object(3)
memory usage: 3.2+ MB
None


In [16]:
print(f'Colunas: {df.columns}')

Colunas: Index(['date', 'rating', 'content', 'product_url'], dtype='object')


In [6]:
display(df.describe())

,rating
count,103476.000000
mean,4.739244
std,0.815946
min,1.000000
25%,5.000000
50%,5.000000
75%,5.000000
max,5.000000


In [17]:
import re
import nltk
import spacy
from nltk.corpus import stopwords
from unidecode import unidecode

## Limpeza Inicial e Pré-processamento
Este notebook trabalha com foco na limpeza inicial e pré-processamento (remoção de stopwords, tokenização)

In [18]:
# carregar modelo de português
nltk.download('stopwords')
nlp = spacy.load("pt_core_news_sm")
stop_words = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\curso\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [25]:
def limpar_texto(texto):
    if pd.isna(texto):
        return ""

    texto = texto.lower()
    texto = unidecode(texto)
    texto = re.sub(r'http\S+|www\S+|@\S+', '', texto)
    texto = re.sub(r'[^a-z\s]', '', texto)

    return texto


In [26]:
df['content_limpo'] = df['content'].apply(limpar_texto)

In [28]:
texts = df['content_limpo'].tolist()

docs = nlp.pipe(
    texts,
    batch_size=1000,
    n_process=1
)


In [29]:
stop_words.discard('nao')
stop_words.discard('não')
stop_words.discard('nunca')

conteudos_processados = []

for doc in docs:
    tokens = [
        token.lemma_
        for token in doc
        if token.text not in stop_words
        and len(token.text) > 2
    ]
    conteudos_processados.append(" ".join(tokens))


In [31]:
conteudo_limpo = df['content_processado'] = conteudos_processados

In [36]:
display(df['content_limpo'].head())

0                                 top
1    produto bom cumpre o que promete
2                     otima qualidade
3                                 bom
4         atendeu minhas expectativas
Name: content_limpo, dtype: object